#**Практическая работа «Анализ на основе RNN»**

***Землянова Анна Группа 932001***

*Задание:*

Необходимо провести повторный анализ текста, который использовался в работе «Предобработка текста », но с использованием import rnnmorph и без использования import pymorphy2.

Если полученные результаты различаются, необходимо пояснить, почему так вышло.

In [4]:
pip install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 59.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rnnmorph: filename=rnnmorph-0.4.1-py3-none-any.whl size=19746349 sha256=b3fed05ec276a6f84fd49b440960ff7580a3f7a99533ec244f59b8dc78663e65
  Stored in directory: /root/.cache/pip/wheels/3d/da/6d/75c269badb3cdb41785bb5e878ce0c044ec164e4587972dc9a
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24620 sha256=deb805942ac7906d20690f74992a78c386d6dee2858e93828202686e9859c04f
  Stored in directory: /root/.cache/pip/wheels/d1/f9/53/8609e08636dbe4bad244c50e3a16ab4d824928e7342059784d
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c3e5179be0f

In [5]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [44]:
# Импортируем RNNMorphPredictor и создаем экземпляр:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language='ru')

In [43]:
def parse_morph_tags(tag_str):
    # Если строка пуста, возвращаем значения "Null" для всех атрибутов
    if tag_str == '_':
        return {'Case': 'Null', 'Gender': 'Null', 'Number': 'Null'}

    # Разделяем строку тегов по символу "|"
    tag_parts = tag_str.split('|')

    tag_dict = {}
    # Разбираем каждую часть тега и добавляем в словарь
    for part in tag_parts:
        key, value = part.split('=')
        tag_dict[key] = value

    return tag_dict

In [54]:
# Прочитаем текст из файла
with open('Text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

    print("Текст до:")
    print(text)

# разбиваем текст на предложения
sentences = re.split(r'(?<=[.!?])\s+', text)

#print(sentences)

lem_and_tag = []

for sentence in sentences:
     words = word_tokenize(sentence)
     #print("\n", words)
     analyzed_words = predictor.predict(words)
     lem_and_tag.extend([(word.pos, word.normal_form, word.tag) for word in analyzed_words])





Текст до:
Маленький кот и большая собака пошли гулять в парк. Черный кот и белая собака бегали весело по зеленой траве. Маленькая девочка кормила кота и собаку. Счастливая девочка улыбалась.
['Маленький кот и большая собака пошли гулять в парк.', 'Черный кот и белая собака бегали весело по зеленой траве.', 'Маленькая девочка кормила кота и собаку.', 'Счастливая девочка улыбалась.']

 ['Маленький', 'кот', 'и', 'большая', 'собака', 'пошли', 'гулять', 'в', 'парк', '.']
1/1 [==============================] - 0s 33ms/step

 ['Черный', 'кот', 'и', 'белая', 'собака', 'бегали', 'весело', 'по', 'зеленой', 'траве', '.']
1/1 [==============================] - 0s 34ms/step

 ['Маленькая', 'девочка', 'кормила', 'кота', 'и', 'собаку', '.']
1/1 [==============================] - 0s 29ms/step

 ['Счастливая', 'девочка', 'улыбалась', '.']
1/1 [==============================] - 0s 26ms/step


In [52]:
# Найдем пары соседних слов, удовлетворяющие условиям (совпадают по роду, числу и падежу):

match_pairs = []

for i in range(len(lem_and_tag) - 1):
    pos1, word1, tag1 = lem_and_tag[i]
    pos2, word2, tag2 = lem_and_tag[i + 1]

    tag1_values = tag1.split('|')
    tag2_values = tag2.split('|')

    #print("tag1_values :", tag1_values )
    #print("tag2_values :", tag2_values, "\n")

    tag1_dict = parse_morph_tags(tag1)
    tag2_dict = parse_morph_tags(tag2)

    if 'NOUN' in pos1 or 'NOUN' in pos2:
        if 'ADJ' in pos1 or 'ADJ' in pos2:
            if tag1_dict['Case']  == tag2_dict['Case']  and tag1_dict['Number'] == tag2_dict['Number'] and tag1_dict['Gender'] == tag2_dict['Gender']:
                match_pairs.append((word1, word2))
                #print(word1, word2)

In [53]:
# Выводим найденные пары:
for pair in match_pairs:
    print(pair[0], pair[1])

маленький кот
больший собака
чёрный кот
белый собака
зелёный трава
маленький девочка
счастливый девочка
